In [18]:
from kahnfigh import Config
from kahnfigh.utils import IgnorableTag

In [19]:
def insert_yaml(x, special_tags=None, global_config={}):
    yaml_path = x.split('!yaml ')[-1]
    inserted_config = Config(yaml_path, special_tags=special_tags)
    global_config.update(inserted_config.get('global', {}))
    if 'global' in inserted_config:
        inserted_config.pop('global')
    return process_tags(inserted_config, global_config)

def identity(x, special_tags=None, global_config={}):
    return x

def replace_var(x, special_tags=None, global_config={}):
    var_name = x.split('!var ')[-1]
    return global_config.get(var_name, x)

ignorable_tags = {'yaml': insert_yaml,
                  'no-cache': identity,
                  'var': replace_var}

def process_tags(conf, global_config):
    for tag_name, tag_processor in ignorable_tags.items():
        tag_paths = conf.find_path('!{}'.format(tag_name), mode='startswith')
        for p in tag_paths:
            conf[p] = tag_processor(conf[p], 
                                    special_tags=[IgnorableTag('!{}'.format(tag)) for tag in ignorable_tags],
                                    global_config = global_config)
    return conf
        
conf = Config('configs/modular_configs/main.yaml',
              special_tags=[IgnorableTag('!{}'.format(tag)) for tag in ignorable_tags])

process_tags(conf,conf.get('global',{}))

{'global': {'var1': 32, 'var2': 42, 'var3': 656, 'var4': 666}, 'TaskA': {'config': {'foo': 5, 'bar': 7, 'properties': {'blabla': 'hello', 'goodbye': 35, 'more_properties': {'lalala': 2, 'lala': 7, 'var2': 42}}}, 'a_magic_number': 32}}

In [15]:
conf

{'global': {'var1': 32, 'var2': 42}, 'TaskA': {'config': None, 'a_magic_number': 32}}